In [1]:
#imports
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os
#os.sys.path

import numpy as np


import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
import glob
from PIL import Image
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

from numpy import random
from sklearn.model_selection import train_test_split
import sys

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn import tree
from sklearn.tree import export_graphviz
from IPython.display import Image
from graphviz import Digraph

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from math import sqrt

from sklearn.neighbors import RadiusNeighborsClassifier

from sklearn.svm import SVC

from sklearn.ensemble import VotingClassifier

In [2]:
#read training and test data
trainset = pd.read_csv("Training.csv")
testset = pd.read_csv("Testing.csv")

In [3]:
#label encoder to assign numeric values to prognosis column 
le = LabelEncoder()
trainset['prognosis_encoded'] = le.fit_transform(trainset.prognosis)
y = trainset['prognosis_encoded'].values
x = trainset.drop(columns=['prognosis_encoded', 'prognosis']).values

In [4]:
#seperate train and validation data
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size = 0.2,stratify = y,random_state = 45)

In [5]:
#view training dataset
print(x)

[[1 3 4 ... 0 0 0]
 [0 3 4 ... 0 0 0]
 [1 0 4 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 3 0 ... 0 0 0]
 [0 3 0 ... 4 2 3]]


In [6]:
#label encoder to assign numeric values to prognosis column for testset
testset['prognosis_encoded'] = le.fit_transform(testset.prognosis)
target = testset['prognosis']
y_test = testset['prognosis_encoded'].values
x_test = testset.drop(columns=['prognosis_encoded', 'prognosis']).values

In [7]:
#logistic regression model
model = LogisticRegression(random_state=42, C=0.1, solver='saga', penalty='l2', multi_class='multinomial', max_iter = 10000)
model.fit(x_train,y_train)

#prediction on validation data
y_pred = model.predict(x_val)

#accuracy score of prediction
acc_score = accuracy_score(y_val, y_pred)
print("val score: ",acc_score)

#cross validation score on full training dataset
scores = cross_val_score(model, x, y, cv=5, n_jobs=1)
print("cross val score",scores)

val score:  1.0
cross val score [1. 1. 1. 1. 1.]


In [8]:
#DISCARDED MODEL
#gaussian naive bayesian network model
nbn = GaussianNB()
nbn.fit(x_train, y_train)

#prediction on validation data
y_pred  =  nbn.predict(x_val)

#accuracy score of prediction
acc_score = accuracy_score(y_val, y_pred)
print("val score: ",acc_score)

#cross validation score on full training dataset
scores = cross_val_score(nbn, x, y, cv=5, n_jobs=1)
print("cross val score",scores)

val score:  1.0
cross val score [1. 1. 1. 1. 1.]


In [9]:
#categorical naive bayesian network model
cbn = CategoricalNB(alpha=0.3)
cbn.fit(x_train, y_train)

#prediction on validation data
y_pred  =  cbn.predict(x_val)

#accuracy score of prediction
acc_score = accuracy_score(y_val, y_pred)
print("val score: ",acc_score)

#cross validation score on full training dataset
scores = cross_val_score(cbn, x, y, cv=5, n_jobs=1)
print("cross val score",scores)

val score:  1.0
cross val score [1. 1. 1. 1. 1.]


In [11]:
#k nearest neighbours classifier model
knn_model = KNeighborsClassifier(n_neighbors=10, p=1, algorithm='ball_tree')
knn_model.fit(x_train, y_train)

#prediction on validation data
y_pred  =  knn_model.predict(x_val)

#accuracy score of prediction
acc_score = accuracy_score(y_val, y_pred)
print("val score: ",acc_score)

#cross validation score on full training dataset
scores = cross_val_score(knn_model, x, y, cv=5, n_jobs=1)
print("cross val score",scores)

val score:  1.0
cross val score [1. 1. 1. 1. 1.]


In [12]:
#SVM soft margin classifier model
svm = SVC(kernel='poly', degree=2, coef0=5, C=5)
svm.fit(x_train, y_train)

#prediction on validation data
y_pred  =  svm.predict(x_val)

#accuracy score of prediction
acc_score = accuracy_score(y_val, y_pred)
print("val score: ",acc_score)

#cross validation score on full training dataset
scores = cross_val_score(svm, x, y, cv=5, n_jobs=1)
print("cross val score",scores)

val score:  1.0
cross val score [1. 1. 1. 1. 1.]


In [13]:
#random forest classifier model
randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)

#predict validation set
y_pred = randomforest.predict(x_val)
acc_score = accuracy_score(y_val, y_pred)
print("Accuracy:", acc_score)
print(randomforest.get_params())

#random search cv to find optimal parameters for random forest
param_dist = {'n_estimators': randint(1,20),
              'max_depth': randint(1,10)}
rand_search = RandomizedSearchCV(estimator = randomforest, 
                                 param_distributions = param_dist, 
                                 n_iter=20, 
                                 cv=5)
rand_search.fit(x_train, y_train)

#get parameters
best_rf = rand_search.best_estimator_
print('Best hyperparameters:',  rand_search.best_params_)
print('Best estimator:',  rand_search.best_estimator_)

Accuracy: 1.0
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Best hyperparameters: {'max_depth': 8, 'n_estimators': 18}
Best estimator: RandomForestClassifier(max_depth=8, n_estimators=18)


In [14]:
#random forest classifier model
rf = RandomForestClassifier(n_estimators=20, max_depth=9)
rf.fit(x_train, y_train)

#prediction on validation data
y_pred  =  rf.predict(x_val)

#accuracy score of prediction
acc_score = accuracy_score(y_val, y_pred)
print("val score: ",acc_score)

#cross validation score on full training dataset
scores = cross_val_score(rf, x, y, cv=5, n_jobs=1)
print("cross val score: ",scores)

#accuracy score of prediction on testset
y_pred_test  =  rf.predict(x_test)
acc_score = accuracy_score(y_test, y_pred_test)
print("test set score: ",acc_score)

val score:  0.9796747967479674
cross val score:  [0.99695122 0.96341463 1.         1.         1.        ]
test set score:  1.0


In [15]:
#voting classifier model
vc = VotingClassifier(estimators=[('lr',LogisticRegression(random_state=42, C=0.1, solver='saga', penalty='l2', multi_class='multinomial', max_iter = 1000)),
                                ('svm',SVC(kernel='poly', degree=2, coef0=5, C=5)),
                                ('rfc',RandomForestClassifier(n_estimators=20, max_depth=9)),
                                ('kkn',KNeighborsClassifier(n_neighbors=10, p=1, algorithm='ball_tree')),
                                ('cbn',CategoricalNB(alpha=0.3))
                               ]
                    )
vc.fit(x_train,y_train)

#prediction on validation data
y_pred  =  vc.predict(x_val)

#accuracy score of prediction
acc_score = accuracy_score(y_val, y_pred)
print("val score: ",acc_score)

#cross validation score on full training dataset
scores = cross_val_score(vc, x, y, cv=5, n_jobs=1)
print("cross val score: ",scores)

#accuracy score of prediction on testset
y_pred_test  =  vc.predict(x_test)
acc_score = accuracy_score(y_test, y_pred_test)
print("test set score: ",acc_score)

val score:  1.0
cross val score:  [1. 1. 1. 1. 1.]
test set score:  1.0
